In [3]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
from katonic.ml import MyClient
import numpy as np
from skl2onnx import to_onnx
import onnxruntime as rt

In [4]:
myclient = MyClient()
mlflow = myclient.mlflow
client = myclient.client

In [5]:
def metric(actual, pred):
    acc_score = accuracy_score(actual, pred)
    recall = recall_score(actual, pred, average='weighted')
    precision_scr = precision_score(actual, pred, average='weighted')
    f1_scr = f1_score(actual, pred, average='weighted')
    
    return (
        acc_score,
        recall,
        f1_scr,
        precision_scr
    )

In [11]:
exp_name = "mlflow-test-onnx"
mlflow.set_experiment(exp_name)
exp_details = mlflow.get_experiment_by_name(exp_name)

iris = load_iris()
X, y = iris.data, iris.target
X = X.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y)
clr = RandomForestClassifier()
clr.fit(X_train, y_train)

with mlflow.start_run(run_name=exp_name):
    onx = to_onnx(clr, X[:1])
    with open("rf_iris.onnx", "wb") as f:
        f.write(onx.SerializeToString())
        
    sess = rt.InferenceSession("rf_iris.onnx", providers=["CPUExecutionProvider"])
    input_name = sess.get_inputs()[0].name
    label_name = sess.get_outputs()[0].name
    pred_onx = sess.run([label_name], {input_name: X_test.astype(np.float32)})[0]

    (acc_score, recall, f1_scr, precision_scr) = metric(y_test, pred_onx)

    model_metrics = {
        "accuracy_score": acc_score,
        "recall": recall,
        "f1_score": f1_scr,
        "precision_score": precision_scr
    }
    
    for metric_name, score in model_metrics.items():
        mlflow.log_metric(metric_name, score)

    model_info = mlflow.onnx.log_model(onnx_model=onx, artifact_path="model")
    

/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
